In [ ]:
# in this version, i will combine output of CNN regression models with the demographic data
# CNN model I am currently using is:
# "elementary_227_227_zoom_16_regression_2022_12_31_hidden_32_relu.h5"
# from Transfer_Learning_final_project_06
# Xception with 128 batch size, 100 epochs, 32 hidden tanh units
# training MSE = 0.1398, R^2 = 0.2211
# validation MSE = 0.1423, R^2 = 0.1517
# testing R = 0.4257, R^2 = 0.1812


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential, load_model
import keras_tuner
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fitter import Fitter, get_common_distributions, get_distributions
import random

In [2]:
# load csv
Data = pd.read_csv("C:\\Users\\Joe\\Box\\Wesley\\Learning Analytics\\HUDK 4050\\Final\\Data\\Data_Table_11_elementary.csv")
N = np.shape(Data)[0]

In [3]:
# load images
folderpath = "C:\\Users\\Joe\\Box\\Wesley\\Learning Analytics\\HUDK 4050\\Final\\Data\\google_maps_satellite_227_227_16_elementary\\"
img_height = 227
img_width = 227
Images = np.zeros(shape = (N, img_height, img_width, 3))
TestScores = Data.cs_mn_avg_ol
for i in range(N):
    id = Data.school_code[i]
    im = tf.keras.utils.load_img(folderpath + "school_" + str(id) + ".jpg",
                                 grayscale=False, color_mode="rgb")
    Images[i,:,:,:] = im

In [4]:
# load training, validation, testing indices
data = np.load('elementary_227_227_zoom_16_regression_2022_12_31_hidden_32_relu_variables.npz')
ind = data['ind']
ind_train = data['ind_train']
ind_val = data['ind_val']
ind_test = data['ind_test']
mse_train = data['mse_train']
mse_val = data['mse_val']
del data

x_train = Images[ind_train,:,:,:]
y_train = TestScores[ind_train]
x_val = Images[ind_val,:,:,:]
y_val = TestScores[ind_val]
x_test = Images[ind_test,:,:,:]
y_test = TestScores[ind_test]

print(N)
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_val))
print(np.shape(y_val))
print(np.shape(x_test))
print(np.shape(y_test))
print(ind[0:5]) # should be [14690  1444   351 18769  5916]

18939
(14204, 227, 227, 3)
(14204,)
(2841, 227, 227, 3)
(2841,)
(1894, 227, 227, 3)
(1894,)
[14690  1444   351 18769  5916]


In [5]:
# load model
model = load_model('elementary_227_227_zoom_16_regression_2022_12_31_hidden_32_relu.h5')

In [7]:
# load predictions
data = np.load('Transfer_Learning_Combined_with_Data_Table_01_predictions.npz')
y_train_pred = data['y_train_pred']
h_train_pred = data['h_train_pred']
y_val_pred = data['y_val_pred']
h_val_pred = data['h_val_pred']
y_test_pred = data['y_test_pred']
h_test_pred = data['h_test_pred']
del data

In [15]:
np.shape(h_test_pred)

(1894, 32)

In [16]:
# add ind to data frame
Data['ind_train'] =  np.zeros(shape = np.shape(ind))
Data['ind_train'][ind_train] = 1
Data['ind_val'] =  np.zeros(shape = np.shape(ind))
Data['ind_val'][ind_val] = 1
Data['ind_test'] =  np.zeros(shape = np.shape(ind))
Data['ind_test'][ind_test] = 1

C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [18]:
# add predictions to data frame
Data['cs_mn_avg_cnn_pred'] = np.zeros(shape = np.shape(ind))
Data['cs_mn_avg_cnn_pred'][ind_train] = y_train_pred
Data['cs_mn_avg_cnn_pred'][ind_val] = y_val_pred
Data['cs_mn_avg_cnn_pred'][ind_test] = y_test_pred

C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: h

In [24]:
# add hidden layer values to data frame
for i in range(32):
    Data['cs_mn_avg_cnn_h_' + str(i)] = np.zeros(shape = np.shape(ind))
    Data['cs_mn_avg_cnn_h_' + str(i)][ind_train] = h_train_pred[:,i]
    Data['cs_mn_avg_cnn_h_' + str(i)][ind_val] = h_val_pred[:,i]
    Data['cs_mn_avg_cnn_h_' + str(i)][ind_test] = h_test_pred[:,i]

C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Joe\anaconda3\envs\HUDK_4050_final_project\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [25]:
Data.head()

,school_code,lea_name,sch_name,street,city,state,zip,grade_low,grade_high,free_lunch,...,cs_mn_avg_cnn_h_22,cs_mn_avg_cnn_h_23,cs_mn_avg_cnn_h_24,cs_mn_avg_cnn_h_25,cs_mn_avg_cnn_h_26,cs_mn_avg_cnn_h_27,cs_mn_avg_cnn_h_28,cs_mn_avg_cnn_h_29,cs_mn_avg_cnn_h_30,cs_mn_avg_cnn_h_31
0,10000500889,Albertville City,Albertville Elementary School,145 West End Drive,Albertville,AL,35950,3.0,4.0,663,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,10000600877,Marshall County,Douglas Elementary School,Highway 75,Douglas,AL,35964,3.0,5.0,338,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
2,10000600880,Marshall County,Brindlee Mountain Elementary School,2233 Shoal Creek Rd,Arab,AL,35016,3.0,5.0,133,...,0.0,0.0,0.0,0.0,0.781368,0.021429,0.0,0.013780,0.0,0.0
3,10000600887,Marshall County,Brindlee Mountain Primary School,1050 Scant City Road,Guntersville,AL,35976,NaN,2.0,149,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.535114,0.0,0.0
4,10000601685,Marshall County,Kate D Smith DAR Elementary School,6077 Main St,Grant,AL,35747,NaN,4.0,226,...,0.0,0.0,0.0,0.0,1.316931,0.000000,0.0,0.000000,0.0,0.0


In [27]:
list(Data.columns) 

['school_code',
 'lea_name',
 'sch_name',
 'street',
 'city',
 'state',
 'zip',
 'grade_low',
 'grade_high',
 'free_lunch',
 'cheap_lunch',
 'pk',
 'kg',
 'g01',
 'g02',
 'g03',
 'g04',
 'g05',
 'g06',
 'g07',
 'g08',
 'g09',
 'g10',
 'g11',
 'g12',
 'total',
 'native',
 'hispanic',
 'black',
 'white',
 'islander',
 'two_or_more',
 'num_teachers',
 'lat',
 'lon',
 'locale_code',
 'county',
 'student_teacher_ratio',
 'title_1_eligible',
 'title_1',
 'native_male',
 'native_female',
 'asian_male',
 'asian_female',
 'hispanic_male',
 'hispanic_female',
 'black_male',
 'black_female',
 'white_male',
 'white_female',
 'islander_male',
 'islander_female',
 'two_or_more_male',
 'two_or_more_female',
 'total_male',
 'total_female',
 'ungraded_students',
 'adult_students',
 'school_level',
 'asian',
 'urbanicity',
 'locale',
 'sch_sped',
 'totenrl',
 'perwht',
 'pernam',
 'perasn',
 'perhsp',
 'perblk',
 'perfl',
 'perrl',
 'perfrl',
 'perecd',
 'gifted_tot',
 'disab_tot',
 'disab_tot_idea',
 '

In [35]:
# save data frame
Data.to_csv('Data_Table_11_elementary_with_cnn.csv', encoding='utf-8', index=False)

In [39]:
idx = Data['ind_train']==1
idx = Data['ind_val']==1
idx = Data['ind_test']==1

R = np.corrcoef(Data['cs_mn_avg_ol'][idx], Data['cs_mn_avg_cnn_pred'][idx])
print(R)
print(R[0,1]*R[0,1])

[[1.         0.42572848]
 [0.42572848 1.        ]]
0.18124474207011995


In [36]:
var1 = Data.perwht[ind_test]
var2 = Data.cs_mn_avg_ol[ind_test]

nanind = np.logical_or(np.isnan(var1),np.isnan(var2))

R = np.corrcoef(var1[~nanind], var2[~nanind])
print(R)
print(R[0,1]*R[0,1])

[[1.         0.57206988]
 [0.57206988 1.        ]]
0.3272639486306846
